In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd 
import numpy as np
from numpy.random import default_rng
import risktools as rt
from time import time

In [2]:
# x = np.random.normal(size=(1000000))
# mu = np.repeat([4.], repeats=1000000)

In [4]:
# rt.extensions.csimOU(x, theta=1., mu=mu, dt=1/12, sigma=0.2, rows=100, cols=1000, log_price=0)

In [3]:
# %%timeit
rt.simOU(log_price=False, sims=50000, T=10, c=True)

Half-life of theta in days =  87.3365447505531


,0,1,2,3,4,5,6,7,8,9,...,49990,49991,49992,49993,49994,49995,49996,49997,49998,49999
0,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
1,4.978782,5.045621,5.040320,4.945172,4.931055,4.970646,5.082893,5.079598,5.062932,5.010821,...,5.078412,4.980620,4.916631,5.037852,5.060802,5.078016,5.090088,4.962437,4.935140,4.937027
2,5.001243,4.940858,4.982681,5.005281,4.929103,4.926161,4.966686,4.914918,5.036039,4.953029,...,5.160349,5.012005,4.884119,4.977073,5.036702,5.061083,5.177767,4.977365,4.927858,5.002532
3,4.960934,4.970218,4.980586,4.904260,4.926154,4.985015,4.936555,4.985470,4.974503,5.075179,...,5.152089,5.047272,4.848339,4.967500,5.145997,5.141886,5.197225,4.944098,4.848336,4.959371
4,4.860502,5.026935,4.992692,4.952875,4.855438,5.084640,4.860124,5.014640,4.909528,5.075650,...,5.192335,5.079358,4.787896,4.951805,5.105593,5.148462,5.137405,4.861253,4.825089,5.038710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516,4.148234,4.446326,3.824242,3.600028,3.462305,3.442209,4.683109,4.678353,4.005137,3.357624,...,4.613259,4.197557,4.177812,3.928506,4.577889,3.616754,3.839033,3.691238,3.640563,4.588025
2517,4.039219,4.511093,3.921304,3.614952,3.441304,3.445001,4.724809,4.481608,4.062323,3.406037,...,4.588997,4.216415,4.224762,3.965512,4.595896,3.605424,3.755403,3.728583,3.611989,4.601325
2518,3.949233,4.513203,3.895505,3.499309,3.340582,3.389833,4.731001,4.415191,4.088493,3.383140,...,4.620345,4.064108,4.078314,4.025830,4.504515,3.567751,3.775535,3.759693,3.666993,4.600936
2519,3.962148,4.498983,3.923081,3.505124,3.360574,3.308562,4.808957,4.319926,4.079457,3.326256,...,4.591390,4.105399,3.946153,3.842152,4.555855,3.492245,3.699105,3.769058,3.568175,4.532888


In [7]:
%%timeit
rt.simOU(log_price=False, sims=50000, T=10, c=False)

Half-life of theta in days =  87.3365447505531
Half-life of theta in days =  87.3365447505531
Half-life of theta in days =  87.3365447505531
Half-life of theta in days =  87.3365447505531
Half-life of theta in days =  87.3365447505531
Half-life of theta in days =  87.3365447505531
Half-life of theta in days =  87.3365447505531
Half-life of theta in days =  87.3365447505531
3.98 s ± 152 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
# x = rt.simGBM_MV(s0=[5,5], r=0.01, sigma=[0.2,0.2], T=1, dt=1/252, cor=[[1,0],[0,1]], sims=100)
# pd.DataFrame(x[:,:,0]).plot(legend=False)